In [2]:
import pandas as pd
import ast
from sklearn.metrics import precision_score, recall_score, f1_score

In [3]:
def safe_literal_eval(cell_str):
    try:
        return ast.literal_eval(cell_str)
    except (SyntaxError, ValueError, TypeError):
        return None

In [4]:
def compute_model_correct_percentages(df, solution_col):
    metadata_columns = {"tokens", "ID", "ner_tags", "collection", "Source"}
    predict_columns = [col for col in df.columns if col not in metadata_columns]
    
    results = {}
    total_rows = len(df)
    
    for col in predict_columns:
        mismatch_count = 0
        skipped_count = 0
        
        for _, row in df.iterrows():
            list_solution = safe_literal_eval(row[solution_col])
            list_pred = safe_literal_eval(row[col])
            
            if list_solution is None or list_pred is None:
                skipped_count += 1
                continue
            
            if len(list_solution) != len(list_pred):
                mismatch_count += 1
        
        usable_rows = total_rows - skipped_count

        if usable_rows == 0:
            correct_percentage = None  
        else:
            correct_percentage = ((usable_rows - mismatch_count) / usable_rows) * 100
        
        results[col] = correct_percentage
    
    return results

datasets = [
    ("Dataset1", "PET-Customized-1.csv"),
    ("Dataset2", "PET-Customized-2.csv"),
    ("Dataset3", "PET-Customized-3.csv")
]

dataset_results = {}

In [5]:
for name, path in datasets:
    df = pd.read_csv(path)
    dataset_results[name] = compute_model_correct_percentages(df, solution_col="ner_tags")

all_models = set()
for result in dataset_results.values():
    all_models.update(result.keys())

table_rows = []

In [6]:
for model in sorted(all_models):
    row_data = {"Model": model}
    values = []  
    for dataset_name, _ in datasets:
        value = dataset_results[dataset_name].get(model)
        row_data[dataset_name] = value
        if value is not None:
            values.append(value)
    row_data["Average"] = sum(values) / len(values) if values else None
    table_rows.append(row_data)

In [7]:
results_df = pd.DataFrame(table_rows).set_index("Model")
print(results_df)

                              Dataset1   Dataset2   Dataset3    Average
Model                                                                  
NER_Tag_Claude3.0-Haiku       0.719424   0.479616   0.719424   0.639488
NER_Tag_Claude3.5-Haiku      12.470024  11.270983  11.990408  11.910472
NER_Tag_Claude3.5-Sonnet     36.450839  35.731415  35.491607  35.891287
NER_Tag_DeepSeek_Chat        20.298507  19.306931  19.753086  19.786175
NER_Tag_GPT4o                19.184652  17.505995  18.705036  18.465228
NER_Tag_GPT4o-mini            7.913669  10.311751   8.633094   8.952838
NER_Tag_Grok2                13.189448   5.995204  11.510791  10.231815
NER_Tag_Llama-3.1-70B         7.434053   6.714628   6.954436   7.034373
NER_Tag_Llama-3.1-8B          5.035971   4.796163   3.597122   4.476419
NER_Tag_gemini-1.0-pro        5.822785   5.665025   6.716418   6.068076
NER_Tag_gemini-1.5-flash      3.399433   2.915452   2.023121   2.779336
NER_Tag_gemini-1.5-flash-8b        NaN   0.000000        NaN   0

In [8]:
def calculate_metrics(df, solution_col):
    relevant_tags = {'B-Activity Data', 'I-Activity Data', 'B-Activity', 
                     'I-Activity', 'B-Actor', 'I-Actor'}
    
    metadata_columns = {"tokens", "ID", "ner_tags", "collection", "source"}
    predict_columns = [col for col in df.columns if col not in metadata_columns]

    results = {}
    for col in predict_columns:
        y_true, y_pred = [], []
        for _, row in df.iterrows():
            solution_list = safe_literal_eval(row[solution_col])
            predicted_list = safe_literal_eval(row[col])
            
            if solution_list is None or predicted_list is None:
                continue
            
            for sol_tag, pred_tag in zip(solution_list, predicted_list):
                if sol_tag in relevant_tags:
                    y_true.append(sol_tag)
                    y_pred.append(pred_tag)

        if not y_true or not y_pred:
            continue

        precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
        recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
        f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
        results[col] = {"Precision": precision, "Recall": recall, "F1-Score": f1}

    return results

In [9]:
datasets = [
    ("Dataset1", "PET-Customized-1.csv"),
    ("Dataset2", "PET-Customized-2.csv"),
    ("Dataset3", "PET-Customized-3.csv")
]

dataset_metrics = {}

for ds_name, ds_path in datasets:
    df = pd.read_csv(ds_path)
    dataset_metrics[ds_name] = calculate_metrics(df, solution_col="ner_tags")

all_models = set()
for ds in dataset_metrics.values():
    all_models.update(ds.keys())

final_results = {}
for metric in ["Precision", "Recall", "F1-Score"]:
    table = []
    for model in sorted(all_models):
        row = {"Model": model}
        values = []
        for ds_name, _ in datasets:
            ds_result = dataset_metrics.get(ds_name, {}).get(model)
            value = ds_result.get(metric) if ds_result is not None else None
            row[ds_name] = value
            if value is not None:
                values.append(value)
        row["Average"] = sum(values) / len(values) if values else None
        table.append(row)
    final_results[metric] = table

for metric, table in final_results.items():
    print(f"\n{metric} results:")
    df_metric = pd.DataFrame(table).set_index("Model")
    print(df_metric)


Precision results:
                          Dataset1  Dataset2  Dataset3   Average
Model                                                           
NER_Tag_Claude3.0-Haiku   0.142431  0.136358  0.132110  0.136966
NER_Tag_Claude3.5-Haiku   0.192661  0.193727  0.191523  0.192637
NER_Tag_Claude3.5-Sonnet  0.250790  0.248723  0.270297  0.256603
NER_Tag_DeepSeek_Chat     0.249867  0.290657  0.225189  0.255238
NER_Tag_GPT4o             0.265484  0.238808  0.265236  0.256509
NER_Tag_GPT4o-mini        0.167259  0.168819  0.166011  0.167363
NER_Tag_Grok2             0.177945  0.208751  0.174461  0.187053
NER_Tag_Llama-3.1-70B     0.158303  0.161902  0.159263  0.159823
NER_Tag_Llama-3.1-8B      0.154176  0.159944  0.153740  0.155953
NER_Tag_gemini-1.0-pro    0.058819  0.072181  0.070845  0.067282
NER_Tag_gemini-1.5-flash  0.206494  0.188993  0.195695  0.197060
NER_Tag_gemini-1.5-pro    0.219225  0.238858  0.218918  0.225667

Recall results:
                          Dataset1  Dataset2  Dataset